In [1]:
from IPython.display import IFrame, Image, display
import numpy as np
import math
import zarr
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import xarray as xr
import torch
import torch.nn as nn
import pandas as pd
from scipy.signal import savgol_filter
import gc
import imageio
from io import BytesIO
from rasterio import features
from affine import Affine
from functions import *


In [ ]:


zarr_path = "../pixel_biomes.zarr/ndvi"

z = zarr.open(zarr_path, mode="r")

ds = xr.open_zarr("../../sample_seasonal_cycle_parameter_preds.zarr")
dates = ds["dates"]
dates_pd = pd.to_datetime(dates)




/home/francesco/miniconda3/envs/ndvi/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


In [11]:
all_dfs = []

for i in np.arange(0,199):

    ndvi_series = z[i, :]

    ndvi_series = ndvi_series.astype(float) / 10000.0


    ndvi_series = np.where((ndvi_series > 1) | (ndvi_series < 0), np.nan, ndvi_series)

    

    df = pd.DataFrame({
        "pixel_id": i,
        'date': dates_pd,
        'ndvi': ndvi_series
        })

    df_sorted = df.sort_values(by='date')

    all_dfs.append(df_sorted)

    # update csv

final_df = pd.concat(all_dfs, ignore_index=True)

final_df.to_csv("ndvi_pixels.csv", index=False)
